Labsheet 1
=========

### Chapter 1 & 2 Fuckaround

# Chapter 1
TBD

# Chapter 2

Housing Values and stuff


In [16]:
# We'll just dump all the imports here, they'll all share the same namespace anyway
import os
import shutil
import tarfile
import urllib
from zlib import crc32

import numpy as np
import pandas as pd

## Error Measures and Notations

$\textbf{Root Mean Square Error}$ (RMSE, aka Root Mean Square Deviation or RMSD) is a commonly used performance measure for a regression analysis. It measures the standard deviation of the errors made in the system's predictions.

$$\text{RMSE}(\mathbf{X},h) = \sqrt{\frac{1}{m} \sum_{i=1}^{m} \Big( h(\mathbf{x}^i) - y^i \Big)^2}$$

* $m$ Number of instances in the dataset.
* $\textbf{x^i}$ is a vector containgin all the feature values for a given instance.
* $\textbf{X}$ is a matrix containing all the feature values of all instances contained within the dataset.
* $y^i$ is the value being predicted by the system (aka the $\mathit{label}$)

* $h$ is the system's prediction function (aka the' $\mathit{hypothesis}$)

## Getting the data

In [17]:
# Make this a function because we really only want to run it once
URL_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
DIR_NAME = "datasets/housing/"

def fetch_housing_data():
    dir_path = DIR_NAME
    if not os.path.isdir(dir_path):
        os.makedirs(dir_path)
    housing_path = DIR_NAME + "housing.tgz"
    with open(housing_path, 'wb') as archive_file:
        data_url = URL_ROOT + DIR_NAME + "housing.tgz"
        with urllib.request.urlopen(data_url) as response:
            shutil.copyfileobj(response, archive_file)
    with tarfile.open(housing_path) as archive_file:
        archive_file.extractall(dir_path)

def load_housing_data():
    csv_path = DIR_NAME + "housing.csv"
    return pd.read_csv(csv_path)

In [21]:
#fetch_housing_data()
housing = load_housing_data()
print(housing.head())

   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEAR BAY  
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2       496.0       177.0         7.2574            352100.0        NEAR BAY  
3       558.0       219.0         5.6431            341300.0        NEAR BAY  
4       565.0       259.0         3.8462            342200.0        NEAR BAY  


In [20]:
# Take a look at the data
housing.head()
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20433 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_value    20640 non-null float64
ocean_proximity       20640 non-null object
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


# Create a Test Set

To avoid overfitting the training model it's important not to evaluate a system with the same data that was used to train it. The simplest solution is to split the dataset into a training and a validation set. The textbook says 80-20 is not a bad split.

In [ ]:
def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

housing_with_id = housing.reset_index()   # adds an `index` column
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")